In [ ]:
import random
import itertools
import numpy as np
import pandas as pd

### params
n_transactions = 40
phi = 0.5
decay = 0.5
beta = 0.5
uncertainity = 0.001
filename = 'trust3.csv'

##setting up
interactions2 = [(random.randrange(100, 105), random.randrange(100, 105), random.uniform(0, 1), random.uniform(0, 1)) for i in range(n_transactions)]

##cleaning repetion
dt = pd.DataFrame(interactions2, columns=['Consumer', 'Producer', 'Consumerrating', 'Producerrating'])
dt = dt[dt['Consumer'] != dt['Producer']]
dt.reset_index(drop=True, inplace=True)
dt = dt.sort_values('Consumer', ignore_index=True)

##reomving same ids
uniqueid1list = dt.Consumer.unique().tolist()
print(uniqueid1list)
dn = pd.DataFrame()
indexlist = []
for i in uniqueid1list:
    for index,row in dt.iterrows():
        row = row.copy()
        if row['Producer'] == i: #and index not in indexlist
            dt.loc[index, 'Producer'] = row['Consumer']
            dt.loc[index, 'Consumer'] = row['Producer']

#attack scenario
scenario = 1#2, 3, 4
no_of_fake_feedback = 50
#get the first row users
dt.head(1).Consumer.item()
for i in range(no_of_fake_feedback):
    if scenario == 1:
        new_row = {'Consumer': dt.head(1).Consumer.item(), 'Producer':dt.head(1).Producer.item(), 'Consumerrating':1, 'Producerrating':random.uniform(0, 1)}
    elif scenario == 2:
        new_row = {'Consumer': dt.head(1).Consumer.item(), 'Producer':dt.head(1).Producer.item(), 'Consumerrating':0.001, 'Producerrating':random.uniform(0, 1)}
    elif scenario == 3:
        new_row = {'Consumer': dt.head(1).Consumer.item(), 'Producer':dt.head(1).Producer.item(), 'Consumerrating':1, 'Producerrating':1}
    elif scenario == 4:
        new_row = {'Consumer': dt.head(1).Consumer.item(), 'Producer':dt.head(1).Producer.item(), 'Consumerrating':random.uniform(0, 1), 'Producerrating':0.001}
        
    dt = dt.append(new_row, ignore_index=True)

##seeing the data
user = 'Consumer'
if user == 'Consumer':
    vd = dt.groupby(['Consumer', 'Producer'])
for k, uid in vd:
    print(uid)

##func one
def get_uid(vd, phi, decay, user):
    newDF = pd.DataFrame()
        
    for k, uid in vd:
        itr_list = uid[user+'rating'].tolist()
#         print(itr_list)
        cum_rj = 0
        cum_sj = 0
        exp_val_n = len(itr_list)
        count = 0
        sj_list = []
        rj_list = []
        cum_sj_list = []
        cum_rj_list = []
        
#         for i in itr_list:
        print('=========================================================')
        for index, row in uid.iterrows():
            count += 1
            rj = ((1- row[user+'rating']) * phi) * (decay **(exp_val_n - count)) #disatisfaction
            sj = ((1+ row[user+'rating']) * phi) * (decay **(exp_val_n - count)) #satisfaction
            cum_rj += rj
            cum_sj += sj
            

#             print('rj ='+ str(rj)+', sj = '+str(sj))
            sj_list.append(sj)
            rj_list.append(rj)
            cum_sj_list.append(cum_sj)
            cum_rj_list.append(cum_rj)
            
            
        
#         print('vendor = '+ str(k)+', st ='+ str(rj)+', ds = '+str(sj))
        uid[user+'dis'] = rj_list  #disatisfaction list
        uid[user+'sat'] = sj_list #satisfaction list
        uid[user+'cum_dis'] = cum_rj_list #cummilative disatisfaction list
        uid[user+'cum_sat'] = cum_sj_list #cummilative satisfaction list
        

        print(uid)
        newDF = newDF.append(uid, ignore_index = True)
    return newDF

##setting decay and calling function one

df = get_uid(vd, phi, decay, user)

##getting ready to call again
user = 'Producer'
df.reset_index(drop=True, inplace=True)
# df
cd = df.groupby(['Consumer', 'Producer'])
df = get_uid(cd, phi, decay, user)

###reseting and grouping df
user = 'Consumer'
df.reset_index(drop=True, inplace=True)
df

cd = df.groupby(['Consumer'])

##  function two
def honesty(df, user='Consumer'):
    honDF = pd.DataFrame()
    for k, af in df:
        print(af)
        hon_li = []
        cum_cumsat = 0
        cum_cumdis = 0
        for index, row in af.iterrows():
            hon = 0
            cum_cumsat += row['Producer'+ 'cum_sat']
            cum_cumdis += row['Producer'+ 'cum_dis']
#             hon = cum_cumsat / (cum_cumsat + cum_cumdis)
            hon = 1 - (cum_cumdis / (cum_cumsat + cum_cumdis))
            print('hon')
            print(hon)
            hon_li.append(hon)

        af['honesty'] = hon_li
        print(af)
        honDF = honDF.append(af, ignore_index = True)
    return honDF

##calling honesty
hon_df = honesty(cd) #for adviser

#func three
def bayes(df, user='Consumer'):
    honDF = pd.DataFrame()
    df.reset_index(drop=True, inplace=True)
    cd = df.groupby(['Consumer'])
    for k, af in cd:
        print(af)
        hon_li = []
        cum_cumsat = 0
        cum_cumdis = 0
        for index, row in af.iterrows():
            expectedtrust = 0
            initr = 0
            inits = 0
            cum_cumsat += row['Consumer'+ 'cum_sat']
            cum_cumdis += row['Consumer'+ 'cum_dis']
#             hon = cum_cumsat / (cum_cumsat + cum_cumdis)
            expectedtrust =  (initr + cum_cumsat) / (initr + inits + cum_cumsat + cum_cumdis)
            print('expectedtrust')
            print(expectedtrust)
            hon_li.append(expectedtrust)

        af['bayesT'] = hon_li
        print(af)
        honDF = honDF.append(af, ignore_index = True)
    return honDF

bdf = bayes(hon_df)

#range 0-1
def get_belief_implicit(cd, epsilon, user):
#     tricky here, col.sum would result different than len(col)

    newdDF = pd.DataFrame()
    for k, af in cd:
        print(af)
        

        dbis = []
        bellis = []
        unlis = []
        
        
        
        for index, row in af.iterrows():
            cum_dis = 0
            cum_sat = 0
            total_sat_dit = 0
            den = 0
            
            cum_dis = row[user+'cum_dis'] 
            cum_sat = (row[user+'cum_sat'])


            print('cum_dis')
            print(cum_dis)
            print('cum_sat')
            print(cum_sat)
            print('total')
            total_sat_dit = cum_dis+cum_sat
            den = total_sat_dit + epsilon 
            print(total_sat_dit)
            belief = cum_sat /(den)
            disbelief = cum_dis /(den)
            uncertainity = epsilon / (den)
            
            dbis.append(disbelief)
            bellis.append(belief)
            unlis.append(uncertainity)
 

        af[user+'bel'] = bellis
        af[user+'disbel'] = dbis
        af[user+'unc'] = unlis

        print(af)
        newdDF = newdDF.append(af, ignore_index = True)


    return newdDF

user = 'Consumer'
bdf.reset_index(drop=True, inplace=True)

cd = bdf.groupby(['Consumer'])


df_dbu =get_belief_implicit(cd,uncertainity, user)

def distancematrix(df):
    uniqueid1list = df.Consumer.unique().tolist()
    comb = list(itertools.combinations(uniqueid1list,2))
    distance_matrix = [[0 for i in range(len(uniqueid1list))] for j in range(len(uniqueid1list))]
    sim_matrix = [[0 for i in range(len(uniqueid1list))] for j in range(len(uniqueid1list))]
    print(distance_matrix)
    print(comb)
    print(uniqueid1list)
    
    idxtoid = {}
    idtoidx = {}
    count = 0
    for i in uniqueid1list:
        idxtoid[count] = i
        idtoidx[i] = count #mostly used
        count += 1

    cd = df.groupby(['Consumer', 'Producer'])

    
    distance_li = []
    normalized_dist = []
    
    ##initializing vendor list
    vendors = {}
    cm_vendors = {}
    
    comv_advisors = {}
    for i in uniqueid1list:
        v = []
        for x, af in cd:
            if(x[0]==i):
#                 print(x[1])
                v.append(x[1])
        vendors[i] = v    
    
    

    for j in uniqueid1list:
        com_ad = []
        for k in uniqueid1list:
            if( j!= k):
                print('===================pairs================')
                print('requestor '+str(j)+' vendors ='+str(vendors[j]))
                print('advisor '+str(k)+' vendors ='+str(vendors[k]))
                comm_vendors =  [v for v, u in zip(vendors[j], vendors[k]) if u == v]
                
                print('common vendors ='+str(comm_vendors))
                cum_dist = 0
                meand = 0
                
                print('searchin for vendor entries===')
                for cv in comm_vendors:
                    
                    nonsqrt = 0
                    dist = 0
                    for x, af in cd:
                        if(x[0]==j and x[1]==cv):
                            print('set i entry')
                            a_bel = 0
                            a_disbel = 0
                            a_unc = 0
                            print(x)
#                             print(af.Consumerbel.tail(1))
#                             print(af.Consumerdisbel.tail(1))
#                             print(af.Consumerunc.tail(1))
                            a_bel = af.Consumerbel.tail(1).item()
                            a_disbel = af.Consumerdisbel.tail(1).item()
                            a_unc = af.Consumerunc.tail(1).item()
                        if(x[0]==k and x[1]==cv):
                            print('set j entry')
                            b_bel = 0
                            b_disbel = 0
                            b_unc = 0
                            print(x)
#                             print(af.Consumerbel.tail(1))
#                             print(af.Consumerdisbel.tail(1))
#                             print(af.Consumerunc.tail(1))
                            b_bel = af.Consumerbel.tail(1).item()
                            b_disbel = af.Consumerdisbel.tail(1).item()
                            b_unc = af.Consumerunc.tail(1).item()
            
                    nonsqrt=((a_bel - b_bel)**2) + ((a_disbel - b_disbel)**2) + ((a_unc - b_unc)**2)
                    dist = nonsqrt** (1/2)
                    dist = dist/2
                    cum_dist += dist
                print('searchin for vendor entries===end')
                if len(comm_vendors) > 0:
                    meand = cum_dist/len(comm_vendors)
                    distance_matrix[idtoidx[j]][idtoidx[k]] = meand
                    sim_matrix[idtoidx[j]][idtoidx[k]] = 1 - meand #Note that this would give 0 if no vendor since it initialize with 0
                    cm_vendors[(j,k)] = comm_vendors
                    com_ad.append(k)
                
                print('======mean distance beteween' +str(j) +' and '+str(k)+' = '+str(meand)+'======')
                print('======sim' +str(j) +' and '+str(k)+' = '+str(1 - meand)+'======')
        comv_advisors[j] = com_ad               
        
    return distance_matrix, sim_matrix, idxtoid, idtoidx, cm_vendors, comv_advisors


##calling distance function
d, sim, idx_id, id_idx, commonv, comv_advisors =distancematrix(df_dbu)


# rep function
def reputation3(df, sim, id_idx, idx_id, commonv, comv_advisors, user='Consumer', alpha=0.2):
    uniqueid1list = df.Consumer.unique().tolist()
    uniqueproducersl = df.Producer.unique().tolist()
    rep_matrix = [[0 for i in range(len(uniqueid1list))] for j in range(len(uniqueproducersl))]
    print(commonv)
    
    mf = df.groupby(['Consumer', 'Producer']) 
    pf = df.groupby(['Producer'])

    
    vidx_vid = {}
    vid_vidx = {}
    count = 0
    for i in uniqueproducersl:
        vidx_vid[count] = i
        vid_vidx[i] = count #mostly used
        count += 1
    print(vidx_vid)
    print(vid_vidx)
    
    v_neigbors = {}
    for y, gf in pf:
        print(gf.Consumer.unique())
        v_neigbors[y] = gf.Consumer.unique().tolist()
    

    print('len(v_neigbors)'+str(len(v_neigbors)))
    print('len(uniqueproducersl'+str(len(uniqueproducersl)))

    vra_rep = {} #(vendor requestor advisor) = reputaion
    for vendor in v_neigbors:
#         print(key) 
        for requestor in uniqueid1list:
#             print(v_neigbors[vendor])
            hon = 0
            rep = 0
            hon_count = 0 #this is m in equation
            rep_count = 0 #this in n in equation

            for advisor in v_neigbors[vendor]:

                if((advisor,requestor) in commonv):
                    ab_sim = sim[id_idx[requestor]][id_idx[advisor]]
                    cum_result = 0
                    print('...commvl'+str((advisor,requestor)))
                    for x, af in mf:
                        if(x[0]==advisor and x[1]==vendor):
                            av_bel = af.Consumerbel.tail(1).item()
                            rep += (ab_sim * av_bel)
                            rep_count += 1

                else:
                    print('no common pairs '+ str((advisor,requestor)))
                    print('so we want '+ str(vendor)+' honesty with '+str(advisor))
                
                    for y, jf in mf: #in this loop following condition will occur only once
                        if(y[0]==advisor and y[1]==vendor): 
                            av_bel = jf.Consumerbel.tail(1).item()
                            va_hon = jf.honesty.tail(1).item()
                            hon += (av_bel * va_hon)
                            hon_count += 1
            reputation = 0
            if rep_count > 0:
                reputation = (alpha * rep)/rep_count
            
            if hon_count > 0:
                reputation += ((1 - alpha) * hon)/hon_count
            
            vra_rep[(vendor, requestor)] = reputation
    return vra_rep


##calling repfuction
vendor_requestor_rep = reputation3(df_dbu, sim, id_idx, idx_id, commonv, comv_advisors)
vendor_requestor_rep

##initial trust
##TODO setup trust_propensity for all users
def init_trust(df, rep, beta=0.5):
    requestor_li = df.Consumer.unique().tolist()
    vendor_lis = df.Producer.unique().tolist()
    rv_initT = {}
    for vendor in vendor_lis:
        for requestor in requestor_li:
            trust_propensity = random.uniform(0, 1)
            r = 0
            if (vendor, requestor) in rep:
                r = rep[(vendor, requestor)]
            initT = (beta * trust_propensity ) + ((1-beta) * r)
            rv_initT[(requestor,vendor)] = initT
    return rv_initT

##calling initrust
intTrustm = init_trust(df_dbu, vendor_requestor_rep, beta)


def trust_T(df,initT, beta=0.5):
    mf = df.groupby(['Consumer', 'Producer'])
    tDtDF = pd.DataFrame()
    for k, af in mf:
        print(k)
        print(af)
        trust_li = []
        trust_dash_li = []
        sum_test = []
        t = 0
        trust_propensity = random.uniform(0, 1)
        for index, row in af.iterrows():
            trust = 0
            trust_dash = 0
            sum_t = 0
#             t = row['init_T'+str(user)]
            if (k[1], k[0]) in initT:
                t = initT[(k[1], k[0])]
            b = row['Consumerbel']
            d = row['Consumerdisbel']
            u = row['Consumerunc']
            trust = b + (t * u)
            trust_dash = d + ((1 - t) * u) 
            sum_t = trust + trust_dash
            print('sum_t')
            print(sum_t)
            trust_li.append(trust)
            trust_dash_li.append(trust_dash)
            sum_test.append(sum_t)

        af['ConsumerT'] = trust_li
        af['ConsumerT_dash'] = trust_dash_li
        af['Consumersum_tdt'] = sum_test
        print(af)
        tDtDF = tDtDF.append(af, ignore_index = True)   
    return tDtDF

##final trust
finDF = trust_T(df_dbu, intTrustm)

##saving file
finDF.to_csv(filename, index=False)

